In [2]:
import MetaTrader5 as mt
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
import pandas as pd
from datetime import datetime
import talib

# Initialize and login to MetaTrader 5
if not mt.initialize():
    print("Failed to initialize MT5")
    exit()

login = 51708234
password = "4bM&wuVJcBTnjV"
server = "ICMarketsEU-Demo"
if not mt.login(login, password, server):
    print("Failed to login to MT5")
    mt.shutdown()
    exit()

symbol = "USDCAD"
timeframe = mt.TIMEFRAME_D1

# Retrieve OHLC data from MetaTrader 5
ohlc_data = pd.DataFrame(mt.copy_rates_range(symbol, timeframe, datetime(2022, 1, 1), datetime.now()))
ohlc_data['time'] = pd.to_datetime(ohlc_data['time'], unit='s')
ohlc_data['time'] = ohlc_data['time'].dt.strftime('%Y-%m-%d %H:%M:%S')
df = ohlc_data.rename(columns={'time': 'Date', 'open': 'Open', 'high': 'High', 'low': 'Low', 'close': 'Close'})
df['Date'] = pd.to_datetime(df['Date'])  # Convert 'Date' to datetime objects
df.set_index('Date', inplace=True)

class RsiOscillator(Strategy):
    upper_bound = 70
    lower_bound = 30
    rsi_window = 14

    def init(self):
        close_prices = pd.Series(self.data.Close)
        close_prices.index = pd.to_datetime(close_prices.index)
        
        # Use talib for RSI calculation
        self.daily_rsi = self.I(talib.RSI, close_prices, self.rsi_window)

        # Resample and apply talib RSI directly
        self.weekly_rsi = self.I(lambda x: x.resample('W-FRI').apply(lambda y: talib.RSI(y, self.rsi_window)), close_prices)

    def next(self):
        price = self.data.Close[-1]

        if crossover(self.daily_rsi, self.upper_bound) and self.weekly_rsi[-1] > self.upper_bound:
            self.position.close()

        elif crossover(self.lower_bound, self.daily_rsi) and self.weekly_rsi[-1] < self.lower_bound:
            self.buy(size=100000, sl=price - 0.0074, tp=price + 0.0148)

# Create and run the backtest
bt = Backtest(df, RsiOscillator, cash=10000, commission=.0003, margin=0.01)

# Run optimization
stats = bt.optimize(
    upper_bound=range(50, 85, 5),
    lower_bound=range(10, 45, 5),
    rsi_window=range(10, 30, 2),
    maximize='Equity Final [$]',
    constraint=lambda param: param.upper_bound > param.lower_bound
)

print(stats)

# Shutdown MT5 connection after data retrieval
mt.shutdown()

bt.plot()


c:\Users\Atharv\anaconda3\envs\my_env\lib\site-packages\backtesting\backtesting.py:1488: UserWarning: Searching for best of 490 configurations.
  output = _optimize_grid()


  0%|          | 0/13 [00:00<?, ?it/s]

c:\Users\Atharv\anaconda3\envs\my_env\lib\site-packages\backtesting\_stats.py:122: RuntimeWarning: divide by zero encountered in scalar divide
  s.loc['Sortino Ratio'] = np.clip((annualized_return - risk_free_rate) / (np.sqrt(np.mean(day_returns.clip(-np.inf, 0)**2)) * np.sqrt(annual_trading_days)), 0, np.inf)  # noqa: E501
c:\Users\Atharv\anaconda3\envs\my_env\lib\site-packages\backtesting\_stats.py:122: RuntimeWarning: divide by zero encountered in scalar divide
  s.loc['Sortino Ratio'] = np.clip((annualized_return - risk_free_rate) / (np.sqrt(np.mean(day_returns.clip(-np.inf, 0)**2)) * np.sqrt(annual_trading_days)), 0, np.inf)  # noqa: E501
c:\Users\Atharv\anaconda3\envs\my_env\lib\site-packages\backtesting\_stats.py:122: RuntimeWarning: divide by zero encountered in scalar divide
  s.loc['Sortino Ratio'] = np.clip((annualized_return - risk_free_rate) / (np.sqrt(np.mean(day_returns.clip(-np.inf, 0)**2)) * np.sqrt(annual_trading_days)), 0, np.inf)  # noqa: E501
c:\Users\Atharv\anacon

Start                     2022-01-03 00:00:00
End                       2024-05-31 00:00:00
Duration                    879 days 00:00:00
Exposure Time [%]                   12.759171
Equity Final [$]                   15766.7268
Equity Peak [$]                    15820.7268
Return [%]                          57.667268
Buy & Hold Return [%]                6.905967
Return (Ann.) [%]                   20.081214
Volatility (Ann.) [%]               32.393034
Sharpe Ratio                         0.619924
Sortino Ratio                        1.447356
Calmar Ratio                          1.17095
Max. Drawdown [%]                  -17.149505
Avg. Drawdown [%]                   -9.359261
Max. Drawdown Duration      420 days 00:00:00
Avg. Drawdown Duration       90 days 00:00:00
# Trades                                   23
Win Rate [%]                        47.826087
Best Trade [%]                       1.175627
Worst Trade [%]                     -0.932925
Avg. Trade [%]                    

c:\Users\Atharv\anaconda3\envs\my_env\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
c:\Users\Atharv\anaconda3\envs\my_env\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],


GridPlot(id='p2692', ...)

In [ ]:
strategy.upper_bound

In [ ]:
strategy.lower_bound

In [ ]:
bt.plot()